В этом ноутбуке обучим модель LaBSE распознавать спам в email

In [1]:
model_name = 'labse'
train_dataset_name = 'spam emails'

In [2]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully u

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [4]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_emails.csv', encoding = "ISO-8859-1")
df

Mounted at /content/drive


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [5]:
df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
df.columns = ['DATA_COLUMN', 'IS_SPAM']

In [6]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...
5166,Subject: put the 10 on the ft\r\nthe transport...,0
5167,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,Subject: industrial worksheets for august 2000...,0


In [7]:
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))

In [8]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291 thi...,0
1,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs this deal is to b...,0
...,...,...
5166,Subject: put the 10 on the ft the transport vo...,0
5167,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,Subject: calpine daily gas nomination > > juli...,0
5169,Subject: industrial worksheets for august 2000...,0


In [9]:
df['IS_SPAM'].value_counts()

0    3672
1    1499
Name: IS_SPAM, dtype: int64

In [10]:
df_positive = df[df['IS_SPAM']==1]

In [11]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [12]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [13]:
df_negative_test.shape

(1836, 2)

In [14]:
df_positive_test.shape

(749, 2)

In [15]:
df_positive_test

,DATA_COLUMN,IS_SPAM
2699,Subject: your mother knows you watch girls moa...,1
2702,Subject: 65 % off for all new software . pyrrh...,1
2703,Subject: [ wrenches ] 68 % off dreamweaver mx ...,1
2706,Subject: he reached around and fingered me whi...,1
2709,"Subject: best meds around , paliourg haven ' t...",1
...,...,...
5159,Subject: pictures streamlined denizen ajar cha...,1
5161,Subject: penny stocks are about timing nomad i...,1
5162,Subject: anomaly boys from 3881 uosda apaprove...,1
5164,Subject: slutty milf wants to meet you take th...,1


In [16]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [17]:
df_balanced_test.sample(10)

,DATA_COLUMN,IS_SPAM
4999,Subject: cutoff for processing invoices and ex...,0
3554,Subject: natural gas nomination for 5 / 01 enr...,0
4283,Subject: meet over 1 million girls sigletos me...,1
4236,"Subject: re : fbw , attention smallcap stox pl...",1
4751,Subject: golf i just have to brag on myself as...,0
4504,"Subject: new pictures for faster viewing , i w...",0
2895,Subject: please re - map hpl zone 4 from a / s...,0
4336,Subject: winter cap for entex when you get all...,0
3297,Subject: cornhusker daren - - - ( re : the ema...,0
2909,Subject: investinme . enron . com login inform...,0


In [18]:
df_balanced_test['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [19]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [20]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [21]:
df_balanced_train['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [22]:
df_balanced_train.sample(10)

,DATA_COLUMN,IS_SPAM
1405,Subject: june 2001 wellhead nomination the att...,0
603,Subject: entex revised estimates for 4 / 00 th...,0
1642,Subject: valero marketing & supply { pasadena ...,0
1505,Subject: activation of price limit orders in p...,0
2283,Subject: january - meter 2186 clear lake city ...,0
2318,Subject: enron net works t & e policy to : all...,0
1113,Subject: re : enron / hpl actuals for august 2...,0
810,Subject: re : tenaska iv 10 / 00 we have recei...,0
2056,Subject: feeling down about the slze of your j...,1
1151,Subject: neon discussion november 15 here are ...,0


In [23]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['IS_SPAM'].squeeze()

In [24]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['IS_SPAM'].squeeze()

Работаем с моделью

In [25]:
labse_preprocess = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")


In [26]:
labse_encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

In [27]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = labse_preprocess(text_input)
outputs = labse_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [28]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [29]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [31]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [32]:
history = model.fit(X_train, y_train, epochs=15)

Epoch 1/15
81/81 [==============================] - 38s 319ms/step - loss: 0.3567 - accuracy: 0.8553 - precision: 0.8834 - recall: 0.5768
Epoch 2/15
81/81 [==============================] - 27s 330ms/step - loss: 0.1929 - accuracy: 0.9400 - precision: 0.9304 - recall: 0.8571
Epoch 3/15
81/81 [==============================] - 27s 331ms/step - loss: 0.1506 - accuracy: 0.9501 - precision: 0.9391 - recall: 0.8852
Epoch 4/15
81/81 [==============================] - 26s 327ms/step - loss: 0.1290 - accuracy: 0.9609 - precision: 0.9500 - recall: 0.9132
Epoch 5/15
81/81 [==============================] - 26s 325ms/step - loss: 0.1154 - accuracy: 0.9640 - precision: 0.9530 - recall: 0.9212
Epoch 6/15
81/81 [==============================] - 26s 325ms/step - loss: 0.1048 - accuracy: 0.9656 - precision: 0.9508 - recall: 0.9292
Epoch 7/15
81/81 [==============================] - 26s 326ms/step - loss: 0.0941 - accuracy: 0.9725 - precision: 0.9581 - recall: 0.9466
Epoch 8/15
81/81 [================

In [33]:
y_predicted = model.predict(X_test)
y_predicted


81/81 [==============================] - 27s 331ms/step


array([[1.3374400e-02],
       [1.1002881e-04],
       [3.4718509e-04],
       ...,
       [9.9987745e-01],
       [9.9326283e-01],
       [7.5449586e-01]], dtype=float32)

In [34]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [35]:
y_test

2536    0
2537    0
2539    0
2540    0
2542    0
       ..
5159    1
5161    1
5162    1
5164    1
5170    1
Name: IS_SPAM, Length: 2585, dtype: int64

In [36]:
accuracy_score(y_test, y_predicted)

0.9675048355899419

In [37]:
precision_score(y_test, y_predicted)


0.9380764163372859

In [38]:
recall_score(y_test, y_predicted)

0.9506008010680908

In [39]:
f1_score(y_test, y_predicted)

0.9442970822281166

In [40]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [41]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [42]:
df_results_on_test

,accuracy,precision,recall,f1_score
labse trained on spam emails and tested on spam emails dataset,0.967505,0.938076,0.950601,0.944297


Сохраним обученную модель

In [43]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/labse_trained_on_spam_emails_19_january'

In [44]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/labse_trained_on_spam_emails_19_january


In [45]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [46]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [47]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [48]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [49]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [50]:
df_with_train_quality

,accuracy,precision,recall,f1_score
labse trained on spam emails quality on train dataset,0.97795,0.965054,0.958611,0.961822


In [51]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [52]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [53]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')